In [1]:
#Importing the standard libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib tk

#Importing the solver modules
import system
from obe import obe 
from states import SigmaLevel,PiLevelParity
from obe import Excitation
import time
import warnings
warnings.filterwarnings('ignore')

In [6]:
#Create a system and generate all the states with the rotation quantum number 0 and 2, and all the -ve parity pi state with J values from 1/2 to 5/2
b=system.System([1],['5/2+'],B_field = [0.0,0.0,4600.0],ignore_mF = False)

b.sigma_Hamiltonian.generate_bare()
b.sigma_Hamiltonian.Zeeman.generate_Zeeman()
b.pi_Hamiltonian.generate_bare()
b.pi_Hamiltonian.Zeeman.generate_Zeeman()

#Next diagonalize the Hamiltonian for this system
b.sigma_Hamiltonian.diagonalize()
b.pi_Hamiltonian.diagonalize()

G_global = b.sigma_Hamiltonian.diagonalized_states
GH_global = np.round(b.sigma_Hamiltonian.diagonalized_Hamiltonian,3)
E_global =b.pi_Hamiltonian.diagonalized_states
EH_global = np.round(b.pi_Hamiltonian.diagonalized_Hamiltonian,3)

print("Hamiltonian generated")

b.generate_branching_ratios(G_global,E_global)
BR_global = b.branching_ratios

12
Hamiltonian generated
Pi branching took : 1.3874847888946533 sec
Sigma- branching took : 1.455503225326538 sec
Sigma+ branching took : 1.3720479011535645 sec


In [4]:
G_global[0]

(-0.82+0j) |G = 0.5, N = 1, F1 = 0.5, F = 1.0, mF = 1.0> + 
(-0.29+0j) |G = 0.5, N = 1, F1 = 1.5, F = 1.0, mF = 1.0> + 
(0.5+0j) |G = 0.5, N = 1, F1 = 1.5, F = 2.0, mF = 1.0>

In [5]:
E_global[4]

(-0.58+0j) |J = 2.5+, F1 = 2.5, F = 2.0, mF = 1.0> + 
(0.82+0j) |J = 2.5+, F1 = 2.5, F = 3.0, mF = 1.0>

### BaF 138 R0 transitions

In [20]:
#Reduce the Hamiltonian to current Hilbert space
G  = G_global
GH = GH_global
E  = E_global
EH = EH_global
BR = BR_global

In [21]:
## Drawww energy levels
plt.figure()
ground_mF = [item.states[np.argmax(np.abs(item.amplitude))].mF+np.linspace(-1/4,1/4,3) for item in G]
ground_Energy = np.array([GH[i,i] for i in range(len(G))])
ground_Energy -= np.mean(ground_Energy)
min_ground_energy = np.amin(ground_Energy)
max_ground_energy = np.amax(ground_Energy)

excited_mF = [item.states[np.argmax(np.abs(item.amplitude))].mF+np.linspace(-1/4,1/4,3) for item in E]
excited_Energy = np.array([EH[i,i] for i in range(len(E))])
excited_Energy -= np.mean(excited_Energy) - 1*np.abs(max_ground_energy-min_ground_energy)
excited_Energy += 150
          
#Plot the energy levels
for i in range(len(G)):
    plt.plot(ground_mF[i],ground_Energy[i]+np.zeros(3))
    plt.text(ground_mF[i][1],ground_Energy[i]-8,f"G[{i}]",fontsize = 8)
for j in range(len(E)):
    plt.plot(excited_mF[j],excited_Energy[j]+np.zeros(3))
    plt.text(excited_mF[j][1],excited_Energy[j]-8,f"E[{j}]",fontsize = 8)


In [22]:
## Rewrite the Hamiltonian in the current Hilbert space
H0 = np.zeros((len(G)+len(E),len(G)+len(E)),dtype=np.complex_)
H0[0:len(G),0:len(G)] = GH
H0[len(G):,len(G):] = EH
b.generate_interaction_Hamiltonian(G,E)
Hint_1=np.round(b.interaction_Hamiltonian,3)
n0 = [1/len(G)]*len(G)+[0]*len(E)

In [29]:
Hint_1[0, len(G)+4]

(-0.201+0j)

In [24]:
## Function to compute the integrated fluorescence
Gamma = 2*np.pi*2.7
def r22(detuning,rabi_prime): #rabi_det_params was a field earlier
    
    tsigma = 113/Gamma/4
    field = Excitation(rabi_prime,0,G[0],E[0],detuning,position=0, diameter=tsigma*4,shape = "Gaussian")

    steps=5000
    n=len(E)+len(G)
    r_init = np.zeros(np.shape(H0)).astype(np.complex128)
    for i in range(len(G)):
        r_init[i,i] = 1.0/len(G)
    
    my_obe = obe(field,[G,E],H0,Hint_1,BR)
    _,ans = my_obe.solve(steps,r_init,max_step_size = 1/Gamma,package='Python')
    
    integrated_fluorescence = 0
    for i in range(steps):
        rho_i = ans[i].reshape(n,n)
        integrated_fluorescence += np.sum(np.diag(rho_i)[len(G):n])

    print(detuning,end = ",")
    
    return np.real(integrated_fluorescence)


In [25]:
from joblib import Parallel,delayed
detuning_scan = np.arange(-300,100,2)

for rabi_prime in [2.0]:
    print(rabi_prime)
    fluorescence = Parallel(n_jobs = -1)(delayed(r22)(detuning,rabi_prime) for detuning in detuning_scan)
    array_to_write = np.array([detuning_scan,fluorescence]).T
    fname = f"BaF138_R0_rabi_{rabi_prime}.csv"
    np.savetxt(fname,array_to_write,delimiter=',')
    print("")


2.0



In [27]:
plt.figure()
for rabi_prime in [2.0]:
    fname = f"BaF138_R0_rabi_{rabi_prime}.csv"
    A = np.genfromtxt(fname,delimiter = ",")
    plt.plot(A[:,0],A[:,1]/np.amax(A[:,1]))

-300,-272,-264,-248,-234,-216,-202,-186,-170,-154,-138,-122,-106,-90,-74,-58,-42,-26,-10,6,22,36,52,68,84,-290,-284,-268,-252,-236,-220,-204,-188,-172,-156,-140,-124,-108,-92,-76,-60,-44,-28,-12,4,20,38,54,70,86,-292,-274,-258,-240,-230,-214,-198,-182,-166,-152,-136,-120,-104,-88,-70,-54,-40,-24,-8,8,24,40,56,72,88,-286,-282,-266,-250,-232,-218,-200,-184,-168,-148,-134,-118,-100,-84,-72,-52,-38,-20,-6,12,28,44,60,74,90,-296,-276,-260,-238,-222,-206,-192,-180,-164,-146,-132,-116,-102,-86,-68,-56,-36,-22,-4,10,26,42,58,76,92,-288,-270,-256,-244,-224,-212,-196,-178,-158,-144,-128,-112,-96,-82,-66,-50,-34,-18,-2,14,30,46,62,78,94,-294,-278,-254,-242,-226,-208,-190,-174,-160,-142,-126,-110,-94,-78,-62,-46,-30,-14,2,18,34,50,66,82,98,-298,-280,-262,-246,-228,-210,-194,-176,-162,-150,-130,-114,-98,-80,-64,-48,-32,-16,0,16,32,48,64,80,96,